# Use Case 4: Real-Time Flight Path Visualisation
Aviation analytics requires processing massive streams of real-time positional data. Using openly licensed flight tracking data from adsb.lol, we'll visualise dynamic flight paths across New Zealand airspace on October 11th, 2025, showing aircraft movements as they take off, transit, and land.

## Key Technical Highlights

- **Dataset Scale**: Complete day of flight tracking data for NZ-connected routes
- **Performance: GPU-accelerated rendering of animated flight trajectories
- **Interactivity: Real-time playback of aircraft movements with temporal controls
- **Memory Efficiency: PyArrow's columnar storage for high-frequency positional updates

## What You'll See
This demonstration showcases the complex choreography of New Zealand's aviation network, revealing traffic patterns, route densities, and temporal flows that provide insights for airspace management, infrastructure planning, and transportation analysis through high-performance geospatial visualisation.

## Datasets

### Flight Tracking Data: adsb.lol
Our aviation data comes from adsb.lol - an open-source, publicly available flight tracking platform that provides:

<img src="images/flights.gif" alt="drawing" width="1200"/>

- Coverage: Global ADS-B (Automatic Dependent Surveillance-Broadcast) data
- Licensing: Openly licensed for research and analysis
- Data Quality: Unfiltered, real-time aircraft positional information
- Temporal Resolution: High-frequency updates capturing precise flight paths

In [1]:
from datetime import datetime, timedelta

import pyarrow as pa
import pyarrow.compute as pc
from lonboard import Map, TripsLayer
from pyarrow.parquet import read_table

In [2]:
# Read data
trips = read_table("data/nz-flights-2025-11-03_traces.parquet")

In [3]:
# Filter the dataset to start when planes operate in NZDT (not in the middle of the night!)
start_time = datetime(2025, 11, 3, 1, 0, 0)
start_time_arrow = pa.scalar(start_time).cast(pa.timestamp("ms", tz="UTC"))
filtered_dataset = trips.filter(
    pc.greater_equal(pc.list_element(pc.field("timestamp"), 0), start_time_arrow)
)

In [4]:
# Create a layer
layer = TripsLayer(
    table=filtered_dataset,
    get_timestamps=filtered_dataset["timestamp"],
    width_min_pixels=2,
    get_color=[11, 41, 72],
    trail_length=1000,
    pickable=False,
)

/mnt/c/Users/samwa/foss4g/.venv/lib/python3.10/site-packages/lonboard/traits/_timestamp.py:152: UserWarning: Reducing precision of input timestamp data to 's' to fit into available GPU precision.
  warnings.warn(


In [5]:
# And add the layer to the map
m = Map(
    layer,
    view_state={
        "longitude": 174.783897,
        "latitude": -37.006117,
        "zoom": 9,
    },
)
m

In [6]:
# We can animate the layer to step through each time step!
layer.animate(step=timedelta(minutes=0.5), fps=60)